In [139]:
import gurobipy as grb
from gurobipy import GRB
import scipy.sparse as spr
import numpy as np
import random
import matplotlib.pyplot as plt
#from sympy import symbols, Rational
from IPython.display import display, Math, Markdown
import numpy.ma as ma
import scipy as sp
from scipy import optimize

In [140]:
class OneToOneITU():
    def __init__(self, n_i, m_j, parameters = (None,None) , Temp = 1,  lbs=(0, 0)):
        self.n_i = n_i
        self.m_j = m_j
        self.lb_U, self.lb_V = lbs
        self.T = Temp
        self.α_ij , self.B_ij = parameters

    def D_ij(self, u_i, v_j,i =None, j=None):
        if j == None:
            return self.α_ij[i,:]  * u_i + (1-self.α_ij[i,:] ) * v_j- self.B_ij[i,:]
        if i == None:
            return self.α_ij[:,j]  * u_i + (1-self.α_ij[:,j] ) * v_j- self.B_ij[:,j]
    

    
    def M_i(self,u_i,v_j,i):
        return np.exp(-u_i/self.T) + np.exp(-self.D_ij(u_i, v_j, i = i)/self.T).sum()

    def M_j(self,u_i,v_j,j):
        return np.exp(-v_j/self.T) + np.exp(-self.D_ij(u_i, v_j, j = j)/self.T).sum()




In [159]:
def generilized_IPFP(self,  tol,initial_price= 30, braket = 100):
    v_j = np.ones(self.m_j)* 10
    u_i = np.zeros(self.n_i)
    tol = 1e-9

    epsilon = np.inf

    while epsilon > tol:
        # print(tol)
        v_j_t= np.zeros(self.m_j)
        u_i_t= np.zeros(self.n_i)
        for i in range(self.n_i):
            u_i_t[i]  += optimize.root_scalar(lambda u:  self.M_i(u ,v_j,i) - 1, bracket=[0, 100], method='brentq').root

        for j in range(self.m_j):
            v_j_t[j]  += optimize.root_scalar(lambda v:  self.M_j(u_i ,v,j) - 1, bracket=[0, 100], method='brentq').root
    
        epsilon = np.maximum(np.max(np.abs(u_i - u_i_t)) ,np.max(np.abs(v_j - v_j_t)))
        u_i = u_i_t
        v_j = v_j_t

    mu_ij = np.zeros((self.n_i+1,self.m_j+1))

    for i in range(self.n_i):
        mu_ij[i,:-1] = np.exp( - self.D_ij(u_i[i],v_j,i=i) / self.T)

    mu_ij[:-1,-1] = np.exp( - u_i / self.T)
    mu_ij[-1,:-1] = np.exp( - v_j / self.T)

    return mu_ij, u_i, v_j

OneToOneITU.generilized_IPFP = generilized_IPFP


## Example 1

In [202]:
n = 2
m = 2
A = np.array([[.6,.5],[.5,.6]])
B = A.copy() #np.ones((n,m)
example_mkt = OneToOneITU(n,m,(A,B), Temp= .1)

In [203]:
mu_ij, u_i, v_j = example_mkt.generilized_IPFP(.5)

In [204]:
example_mkt.M_i(u_i[1] ,v_j,1) - 1

-6.106226635438361e-15

In [205]:
example_mkt.M_j(u_i ,v_j[1],1) - 1

-5.697557203809822e-09

In [206]:

x = example_mkt.T * (np.log(np.exp((example_mkt.B_ij[0,0]-example_mkt.B_ij[0,0])/ example_mkt.T ) + np.exp((example_mkt.B_ij[0,1]-example_mkt.B_ij[0,0])/ example_mkt.T ) + np.exp(-example_mkt.B_ij[0,0]/ example_mkt.T)))+ example_mkt.B_ij[0,0]
print(x)
print(example_mkt.M_i(x ,np.array([x,x]),1) - 1)
print(example_mkt.M_j(np.array([x,x]),x, 1) - 1)

0.6315072160665252
-4.440892098500626e-16
-4.440892098500626e-16


In [207]:
print(u_i)
print(v_j)
print(mu_ij)

[0.63150708 0.63150708]
[0.6315074 0.6315074]
[[0.72973627 0.26845489 0.00180884]
 [0.26845489 0.72973627 0.00180884]
 [0.00180883 0.00180883 0.        ]]


In [116]:
mu_ij.sum(0)

array([0.99999999, 0.99999999, 0.00361768])

### Nevertheless

In [11]:
v_j_eq = np.zeros(2)
u_i_eq = np.ones(2) 


In [12]:
print(example_mkt.D_ij(u_i_eq[0],v_j_eq,0))
print(example_mkt.D_ij(u_i_eq[1],v_j_eq,1))


[0. 0.]
[0. 0.]


## Example 2

The solution here does not depend on sigma

In [135]:
n = 2
m = 1
A = np.array([[.6],
              [.5]])
B = np.zeros_like(A) #A.copy() #np.ones((n,m)
example_mkt_2 = OneToOneITU(n,m,(A,B), Temp= .04)

In [136]:
mu_ij, u_i, v_j = example_mkt_2.generilized_IPFP(1e-12)

In [137]:
print(u_i)
print(v_j)
print(mu_ij)

[0.02062466 0.01872839]
[0.05997771]
[[0.40286769 0.59713231]
 [0.3738778  0.62612221]
 [0.22325451 0.        ]]


In [138]:
mu_ij.sum(0)

array([1.        , 1.22325452])

In [29]:
example_mkt.D_ij( u_i[0], v_j,i = 0 )

array([11.23532094, 11.23532094])

In [30]:
example_mkt_2.T

10

In [31]:
mu_ij = np.zeros((example_mkt_2.n_i+1,example_mkt_2.m_j+1))

for i in range(example_mkt_2.n_i):
    mu_ij[i,:-1] = np.exp( - example_mkt_2.D_ij(u_i[i],v_j,i=i) / example_mkt_2.T)

mu_ij[:-1,-1] = np.exp( - u_i / example_mkt_2.T)
mu_ij[-1,:-1] = np.exp( - v_j / example_mkt_2.T)

## Example 3

In [597]:
alpha = 4
n = 3
m = 3
# A = np.ones((3,3))
A = np.array([[1,alpha,alpha],
              [alpha,alpha,1],
              [alpha,1,alpha]]) 

B = np.array([[1,1,0],
              [1,1,1],
              [0,1,2]])

example_mkt = OneToOneITU(n,m,(1/(1+A),B/(1+A)), Temp= .05)

In [598]:
mu_ij, u_i, v_j = example_mkt.generilized_IPFP(1e-13)

In [599]:
example_mkt.M_j(u_i ,v_j[1],1) - 1

-1.1102230246251565e-16

In [600]:
example_mkt.M_i(u_i[1] ,v_j,1) - 1

9.828219349472533e-09

In [601]:
u_i

array([0.5438654 , 0.40431213, 0.58133103])

In [602]:
v_j

array([0.45687925, 0.42074348, 0.59775626])

In [603]:
mu_ij.round(2)

array([[0.99, 0.01, 0.  , 0.  ],
       [0.01, 0.01, 0.98, 0.  ],
       [0.  , 0.98, 0.02, 0.  ],
       [0.  , 0.  , 0.  , 0.  ]])

In [604]:
mu_ij.sum(0)

array([1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 3.3555107e-04])

In [605]:
example_mkt.D_ij(u_i[2],v_j,2)

array([0.48176961, 0.00103726, 0.19447122])

In [606]:
example_mkt.D_ij(.5,np.ones(3)*.5,i = 2)

array([0.5, 0. , 0.1])

In [607]:
example_mkt.B_ij

array([[0.5, 0.2, 0. ],
       [0.2, 0.2, 0.5],
       [0. , 0.5, 0.4]])

In [608]:
A[2,:]* v_j [None,:] + u_i[2] - B[2,:] 

array([[2.40884804e+00, 2.07451075e-03, 9.72356085e-01]])

In [609]:
A[2,:]

array([4, 1, 4])

In [610]:
v_j

array([0.45687925, 0.42074348, 0.59775626])

In [611]:
B[2,:] - A[2,:]* v_j [None,:] 

array([[-1.82751702,  0.57925652, -0.39102506]])